# Imports

In [1]:
import numpy as np
from scipy.ndimage.interpolation import shift

# Utils Functions

In [2]:
def calc_neigbhors_sum(mat, i, j, val):
    return mat[i - 1][j - 1]*val + mat[i - 1][j]*val + mat[i - 1][j + 1]*val \
            + mat[i][j - 1]*val + mat[i][j + 1]*val \
            + mat[i + 1][j - 1]*val + mat[i + 1][j]*val + mat[i + 1][j + 1]*val

# returns prob for +1
def calc_prob(mat, Temp, i,j):
    x_plus = calc_neigbhors_sum(mat, i, j, 1)
    x_minus = calc_neigbhors_sum(mat, i, j, -1)
    Z_temp = np.exp((1/Temp) * x_plus) + np.exp((1/Temp) * x_minus)
    return np.exp((1/Temp) * x_plus)/Z_temp

def sample_site(mat, Temp, i, j):
    prob = calc_prob(mat, Temp, i,j)
    return np.random.choice([1, -1], 1, p=[prob, 1-prob])

def MRF_iteration(mat, Temp, lat_size=8):
    for i in range(lat_size):
        for j in range(lat_size):
            mat[i+1][j+1] = sample_site(mat, Temp, i+1, j+1)
    return mat

def Gibbs_sampler(Temp, iterations, lat_size=8):
    initial_mat = np.random.randint(low=0,high=2,size=(lat_size,lat_size))*2 - 1
    padded_mat = np.pad(initial_mat, ((1,1),(1,1)), 'constant')
    for iteration in range(iterations):
        padded_mat = MRF_iteration(padded_mat, Temp)
    return padded_mat

def compute_empirical_expectation(num_samples, Temp, iterations=25):
    normalizing_factor = num_samples
    sum12 = 0
    sum18 = 0
    for sample_step in range(num_samples):
        if sample_step % 100 is 99:
            print("Step: " + str(sample_step))
        sample = Gibbs_sampler(Temp, iterations)
        sum12 += sample[1][1] * sample[2][2]
        sum18 += sample[1][1] * sample[8][8]
    return sum12 / normalizing_factor, sum18 / normalizing_factor


# Ex 1 Method 1: Empirical expectation of Independent Samples

In [6]:
print("Method 1: Independent Samples")
print(compute_empirical_expectation(1000, 2))

Method 1: Independent Samples
Step: 99
Step: 199
Step: 299
Step: 399
Step: 499
Step: 599
Step: 699
Step: 799
Step: 899
Step: 999
(0.89, 0.658)


# Ergodicity Sampling function

In [11]:
def Gibbs_sampler_ergodicity(Temp, sweeps=25000, lat_size=8, ignore_first_size=100):
    initial_mat = np.random.randint(low=0,high=2,size=(lat_size,lat_size))*2 - 1
    padded_mat = np.pad(initial_mat, ((1,1),(1,1)), 'constant')
    normalizing_factor = sweeps - ignore_first_size
    sum12 = 0
    sum18 = 0
    for sweep in range(sweeps):
        #print('Sweep Number: ' + str(sweep))
        padded_mat = MRF_iteration(padded_mat, Temp)
        if sweep >= ignore_first_size:
            sum12 += padded_mat[1][1] * padded_mat[2][2]
            sum18 += padded_mat[1][1] * padded_mat[8][8]
    return sum12 / normalizing_factor, sum18 / normalizing_factor

# Ex 1 Method 2: Ergodicity expectation


In [12]:
print("Method 2: Ergodicity")
print(Gibbs_sampler_ergodicity(1, sweeps=5000))

Method 2: Ergodicity
(0.9955102040816326, 0.9779591836734693)
